# n = 50 Autoregressive covariance structure 

We try to parameterize covariance Gamma only using two parameters rho and sigma2 as in the AR(1) structure.

For n = 50 this autoregressive model converges in 35 iterations using Quasi-Newton, 21 with Newton.

$$\mu_i= 5, \rho = 0.9, \sigma^2 = 0.1, n_i = 50$$

$$\Gamma_i = \sigma^2 * \begin{pmatrix} 1 & \rho & \rho^2 & \rho^3 & ...  &\rho^{n_i - 1}\\ \rho & 1 & \rho & \rho^2 & ... \\ & & ... & & \\ & &...& \rho & 1 & \rho \\ \rho^{n_i - 1} & \rho^{n_i - 2} & ...& \rho^2 & \rho & 1\end{pmatrix} \forall i \in [1, N = 10,000]$$ 

1. First we need to modify the structure itself
2. Second modify the loglikelihood function 
3. Third modify gradient and hessian with respect to beta
4. ADD gradient and hessian with respect to AR(1) parameterization
5. Quasi-Newton works
6. Newton works with cross terms for: rho and sigma2 and beta and sigma2

In [1]:
using DataFrames, Random, GLM, GLMCopula, Test
using LinearAlgebra, BenchmarkTools, Revise

Random.seed!(1234)

# sample size
N = 10000
# observations per subject
n = 50
ρ = 0.9
σ2 = 0.1

V = zeros(n, n) # will store the AR(1) structure without sigma2

mean = 5

dist = Poisson

V = get_AR_cov(n, ρ, σ2, V)

# true Gamma
Γ = σ2 * V

50×50 Array{Float64,2}:
 0.1          0.09         0.081        …  0.000636269  0.000572642
 0.09         0.1          0.09            0.000706965  0.000636269
 0.081        0.09         0.1             0.000785517  0.000706965
 0.0729       0.081        0.09            0.000872796  0.000785517
 0.06561      0.0729       0.081           0.000969774  0.000872796
 0.059049     0.06561      0.0729       …  0.00107753   0.000969774
 0.0531441    0.059049     0.06561         0.00119725   0.00107753
 0.0478297    0.0531441    0.059049        0.00133028   0.00119725
 0.0430467    0.0478297    0.0531441       0.00147809   0.00133028
 0.038742     0.0430467    0.0478297       0.00164232   0.00147809
 0.0348678    0.038742     0.0430467    …  0.0018248    0.00164232
 0.0313811    0.0348678    0.038742        0.00202756   0.0018248
 0.028243     0.0313811    0.0348678       0.00225284   0.00202756
 ⋮                                      ⋱               
 0.0018248    0.00202756   0.00225284      

In [2]:
vecd = [dist(mean) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

10000-element Array{Array{Float64,1},1}:
 [7.0, 8.0, 7.0, 4.0, 8.0, 2.0, 6.0, 6.0, 6.0, 7.0  …  5.0, 3.0, 4.0, 6.0, 4.0, 5.0, 5.0, 6.0, 4.0, 7.0]
 [3.0, 7.0, 6.0, 7.0, 6.0, 6.0, 3.0, 3.0, 8.0, 5.0  …  3.0, 4.0, 5.0, 8.0, 2.0, 5.0, 4.0, 4.0, 8.0, 6.0]
 [6.0, 3.0, 3.0, 7.0, 4.0, 3.0, 6.0, 2.0, 1.0, 8.0  …  2.0, 4.0, 3.0, 4.0, 6.0, 3.0, 4.0, 4.0, 3.0, 5.0]
 [9.0, 4.0, 5.0, 3.0, 5.0, 5.0, 4.0, 12.0, 4.0, 5.0  …  4.0, 8.0, 7.0, 9.0, 5.0, 3.0, 8.0, 5.0, 4.0, 4.0]
 [5.0, 7.0, 4.0, 7.0, 4.0, 1.0, 7.0, 3.0, 5.0, 4.0  …  9.0, 7.0, 3.0, 1.0, 3.0, 5.0, 3.0, 6.0, 5.0, 5.0]
 [6.0, 6.0, 10.0, 2.0, 4.0, 2.0, 6.0, 7.0, 1.0, 5.0  …  2.0, 6.0, 3.0, 6.0, 3.0, 6.0, 4.0, 4.0, 4.0, 5.0]
 [5.0, 4.0, 6.0, 3.0, 4.0, 6.0, 0.0, 3.0, 3.0, 3.0  …  4.0, 6.0, 3.0, 3.0, 5.0, 1.0, 4.0, 1.0, 5.0, 10.0]
 [6.0, 4.0, 5.0, 7.0, 5.0, 8.0, 5.0, 3.0, 10.0, 1.0  …  5.0, 4.0, 5.0, 3.0, 4.0, 6.0, 5.0, 4.0, 7.0, 2.0]
 [4.0, 5.0, 8.0, 5.0, 5.0, 4.0, 7.0, 3.0, 3.0, 6.0  …  5.0, 2.0, 4.0, 7.0, 6.0, 5.0, 6.0, 5.0, 7.0, 4.0]
 [4.0, 6.0

In [3]:
Random.seed!(1234)

d = Poisson()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaARObs{T, D, Link}}(undef, N)

for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    gcs[i] = GLMCopulaARObs(y, X, d, link)
end

gcm = GLMCopulaARModel(gcs);

In [4]:
initialize_model!(gcm)
@show gcm.β
@show exp.(gcm.β);
fill!(gcm.ρ, 0.0)
fill!(gcm.σ2, 0.0)

loglikelihood!(gcm, true, true)

gcm2 = deepcopy(gcm);

initializing β using Newton's Algorithm under Independence Assumption
1 0.0 -1.1094692620433543e6 39999
2 -1.1094692620433543e6 -1.1094692620433543e6 39999
gcm.β = [1.6135629925799966]
exp.(gcm.β) = [5.020668000000001]


# Quasi-Newton
## Number of Iterations....: 35

## 26 seconds

In [5]:
@time GLMCopula.fit!(gcm, IpoptSolver(print_level = 5, max_iter = 100, tol = 10^-6, hessian_approximation = "limited-memory"))

gcm.θ = [1.6135629925799966, 0.0, 0.0]
gcm.∇θ = [-5.411493475548923e-10, 0.0, 7029.3850228694855]
gcm.θ = [1.6135629925799966, 0.00999999, 0.00999999]
gcm.∇θ = [-2007.9362838296047, 94.36994006980224, 3073.2746573758886]

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:  

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  30  1.1087314e+06 0.00e+00 9.16e-04 -11.0 1.84e-06    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6108349084126021, 0.8961115378168762, 0.09889404966465418]
gcm.∇θ = [0.11693868080790093, -0.0008188942513265296, -0.00011026546914827406]
  31  1.1087314e+06 0.00e+00 1.66e-03 -11.0 1.97e-06    -  1.00e+00 2.50e-01f  3
gcm.θ = [1.6108349607587051, 0.8961115400685955, 0.09889416578769991]
gcm.∇θ = [0.020504250497419108, -0.0028218114837046393, -0.00224795577965331]
  32  1.1087314e+06 0.00e+00 2.92e-04 -11.0 2.32e-07    -  1.00e+00 5.00e-01f  2
gcm.θ = [1.6108349736273309, 0.8961115435172329, 0.09889395904077701]
gcm.∇θ = [-9.333210156903249e-5, 0.00024633823349562434, 3.728846315009804e-5]
  33  1.1087314e+06 0.00e+00 3.50e-06 -11.0 2.07e-07    -  1.00e+00 1.00e+00f  1
gcm.θ = [1.6108349745593853, 0.8961116148785871, 0.09889387516048315]
gcm.∇θ = [-0.0007543102438329896, -0.002044795650786746, -8.106310293776176e-5

-1.108731409241505e6

# Newton
## Number of Iterations....: 21

## 20 seconds

In [6]:
@time GLMCopula.fit!(gcm2, IpoptSolver(print_level = 5, max_iter = 100, tol = 10^-6, hessian_approximation = "exact"))

gcm.θ = [1.6135629925799966, 0.0, 0.0]
gcm.∇θ = [-5.411493475548923e-10, 0.0, 7029.3850228694855]
gcm.θ = [1.6135629925799966, 0.00999999, 0.00999999]
gcm.∇θ = [-2007.9362838296047, 94.36994006980224, 3073.2746573758886]
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        5

Total number of variables............................:        3
                     variables with only lower bounds:        1
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:     

-1.10873140924151e6

In [7]:
@show gcm.∇θ
@show gcm2.∇θ;

gcm.∇θ = [-3.0205193830568078e-6, -9.651683381406428e-7, -8.669954576490113e-7]
gcm2.∇θ = [1.0928347414562722e-5, 5.280776688554312e-5, -8.113131061460166e-5]


In [8]:
@show gcm.θ
@show gcm2.θ;

gcm.θ = [1.6108349736123844, 0.8961115500664648, 0.09889395370042722]
gcm2.θ = [1.6108349735088143, 0.8961115447751725, 0.09889396729215337]


In [9]:
@show loglikelihood!(gcm, true, true)
@show loglikelihood!(gcm2, true, true);

loglikelihood!(gcm, true, true) = -1.108731409241505e6
loglikelihood!(gcm2, true, true) = -1.10873140924151e6
